In [1]:
%load_ext jupyter_black

# Case Study of OOP (Payment Service Integration with Payment Service Provider)

**Problem Description:** 

Design a high level OOP design of a system that handles payments from such as bKash, nagad & also payment service provider like Easten Bank Ltd. Payment gateway and CityBank.

**Functional Requirements:**

- Pay-in-flow: Payment system receives money from customer and make the premium sattelement.

**Non Functional Requirements:**

- Reliability: handle failed payments
- proper logging at each steps
- Follow Clean Architechture
- Modular design

**High-Level Premium Pay Flow**

![psp_diagram](assets/use_case_diagram_psp.png "High-Level Premium Pay Flow")

**Class Entities**

- Payment Service
    - The payment service accepts payment events from users and coordinates the payment process. The first thing is usually does is check of business rules for a single policy object payments.Also it should have a functionlity for generate a unique order id for each payment execution 
- Payment Executor
    - The payment executor executes a single payment order via a payment service provider(PSP).
- Ledger
    - The ledger keeps a financial record of the payment transaction. 
- Insurer
    - The Insurer is the policy object information from the system, where a details of the policy is found.

In [10]:
import math
import random
from typing import Any
from abc import ABC, abstractmethod
from datetime import datetime, date
from enum import Enum


class ServiceProvider(str, Enum):
    MFS = "MFS"
    PG = "PAYMENT_GATEWAY"
    AB = "AGENT_BANKING"


class Insurer:
    """The class for Insurer Policy Handler. It is responsible for the all the busiess logic sanity check"""

    MINIMUM_DUE_DAYS = -90
    MAXIMUM_ADV_PAYDAYS = 20
    MINIMUM_PREMIUM_COUNT = 1
    ALLOWED_LAPSE_MONTH = 3.2

    def __init__(self, lapse_month: float, paymode: str):
        self.lapse_month = lapse_month
        self.paymode = paymode

    @property
    def lapsed(self) -> bool:
        return self.lapse_month > self.ALLOWED_LAPSE_MONTH

    @property
    def term_in_month(self) -> int:
        match self.paymode:
            case "1":
                return 12
            case "2":
                return 6
            case "4":
                return 3
            case _:
                return 1

    def due_count(self) -> int:
        return math.ceil(self.lapse_month / self.term_in_month)

    def due_amount(self) -> float:
        current_date = datetime.today()
        next_premium_date = self.current_next_paydate

        diff_in_days = (next_premium_date - current_date).days
        premium_amount = 0
        if (self.MINIMUM_DUE_DAYS <= diff_in_days <= self.MAXIMUM_ADV_PAYDAYS) or (
            self.MINIMUM_DUE_DAYS > diff_in_days and self.due_count() > 0
        ):
            premium_amount = self.total_premium * max(
                self.due_count(), self.MINIMUM_PREMIUM_COUNT
            )
            suspanse_amount = self.suspanse_amount or 0
        return (premium_amount + self.late_fees) - suspanse_amount

    def get_ledger_info(self) -> dict[str, Any]:
        return {
            "policy_number": "0134050541",
            "lapsed": True,
        }


class AbstractPaymentService(ABC):
    """Abstract class to blueprint the inherited classes should have"""

    @abstractmethod
    def add_ledger_entry(self) -> None:
        raise NotImplementedError


class Order:
    def __init__(self) -> None:
        pass

    def create_orderid(self) -> str:
        return "ORDERID CREATED"


class PaymentService(AbstractPaymentService):
    def __init__(self, db) -> None:
        self.__is_payable = False
        self._orderid = None
        self.db = db

    def create_orderid(self, office_code: str) -> None:
        current_date = date.today().strftime("%y%m%d")

        # Generate random letters without digits
        random_letters = "".join(
            random.choices(string.ascii_uppercase, k=orderid_length - 2)
        )

        # Generate two random digits
        random_digits = "".join(random.choices(string.digits, k=2))

        # Insert the two random digits at random positions within the random_letters
        random_position = random.randint(0, len(random_letters))
        random_letters = (
            random_letters[:random_position]
            + random_digits
            + random_letters[random_position:]
        )

        self._orderid = f"{office_code}{current_date}{random_letters}"

    @property
    def order_id(self) -> str | None:
        return self._orderid

    def add_ledger_entry(self, payload: dict[str, Any]) -> None:
        self.db.add(payload)


class MFSPaymentService(PaymentService):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.service_type = ServiceProvider.MFS


class ABPaymentService(PaymentService):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.service_type = ServiceProvider.AB


class PGPaymentService(PaymentService):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.service_type = ServiceProvider.PG


class PaymentExecutor(PaymentService):
    def cancel_orderid(self) -> str:
        return "ORDERID CANCELLED"

    def reject_orderId(self) -> str:
        return "ORDERID REJECTED"

    def execute(self) -> None:
        pass

In [6]:
insurer = Insurer(3.21, "3")
ps = PaymentService()

print(insurer.lapsed)
print(insurer.term_in_month)
print(insurer.due_count())

True
1
4
